In [2]:
import os
import re
import sqlite3

# Function to parse a single .abs file
def parse_abs_data(abs_data):
    patterns = {
        'paper_id': r'Paper: (.+)',
        'authors': r'Authors: (.+)',
        'title': r'Title: (.+)',
        'comments': r'Comments: (.+)',
        'subj_class': r'Subj-class: (.+)',
        'journal_ref': r'Journal-ref: (.+)',
        'abstract': r'\n\\\\\n ([\s\S]+?)\n\\\\\n',
    }
    return {key: re.search(pattern, abs_data).group(1).strip() if re.search(pattern, abs_data) else None
            for key, pattern in patterns.items()}

# Relational Database Uploader class
class RelationalDatabaseUploader:
    def __init__(self, db_path):
        self.connection = sqlite3.connect(db_path)
        self.cursor = self.connection.cursor()
        self._create_tables()

    def _create_tables(self):
        # Create tables for papers and citations
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS papers (
            paper_id TEXT PRIMARY KEY,
            title TEXT,
            authors TEXT,
            comments TEXT,
            subj_class TEXT,
            journal_ref TEXT,
            abstract TEXT
        )""")
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS citations (
            from_paper_id TEXT,
            to_paper_id TEXT,
            PRIMARY KEY (from_paper_id, to_paper_id),
            FOREIGN KEY (from_paper_id) REFERENCES papers(paper_id),
            FOREIGN KEY (to_paper_id) REFERENCES papers(paper_id)
        )""")
        
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS paper_dates (
            paper_id TEXT PRIMARY KEY,
            publication_date DATE NOT NULL,
            FOREIGN KEY (paper_id) REFERENCES papers(paper_id)
        )""")
        
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS co_citations (
            paper_1_id TEXT,
            paper_2_id TEXT,
            co_citation_count INTEGER,
            PRIMARY KEY (paper_1_id, paper_2_id),
            FOREIGN KEY (paper_1_id) REFERENCES papers(paper_id),
            FOREIGN KEY (paper_2_id) REFERENCES papers(paper_id)
        )""")

        self.connection.commit()

    def insert_paper(self, parsed_data):
        query = """
        INSERT OR REPLACE INTO papers (paper_id, title, authors, comments, subj_class, journal_ref, abstract)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        self.cursor.execute(query, (
            parsed_data.get('paper_id'),
            parsed_data.get('title'),
            parsed_data.get('authors'),
            parsed_data.get('comments'),
            parsed_data.get('subj_class'),
            parsed_data.get('journal_ref'),
            parsed_data.get('abstract'),
        ))
        self.connection.commit()

    def insert_citation(self, from_node, to_node):
        query = """
        INSERT OR IGNORE INTO citations (from_paper_id, to_paper_id)
        VALUES (?, ?)
        """
        self.cursor.execute(query, (from_node, to_node))
        self.connection.commit()
        
    def insert_paper_date(self, paper_id, publication_date):
        query = """
        INSERT OR IGNORE INTO paper_dates (paper_id, publication_date)
        VALUES (?, ?)
        """
        self.cursor.execute(query, (paper_id, publication_date))
        self.connection.commit()
    
    
    def insert_co_citation(self, paper_1_id, paper_2_id, co_citation_count):
        query = """
        INSERT OR REPLACE INTO co_citations (paper_1_id, paper_2_id, co_citation_count)
        VALUES (?, ?, ?)
        """
        self.cursor.execute(query, (paper_1_id, paper_2_id, co_citation_count))
        self.connection.commit()

    def close(self):
        self.connection.close()

# Function to process all .abs files and create rows in the database
def process_all_abs_files(root_folder, db_uploader):
    for year_folder in os.listdir(root_folder):
        year_path = os.path.join(root_folder, year_folder)
        if os.path.isdir(year_path) and year_folder.isdigit():
            for file_name in os.listdir(year_path):
                if file_name.endswith(".abs"):
                    file_path = os.path.join(year_path, file_name)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        abs_data = file.read()
                        parsed_data = parse_abs_data(abs_data)
                        # Remove leading zeros from filename
                        parsed_data['paper_id'] = str(int(file_name.replace('.abs', '')))
                        db_uploader.insert_paper(parsed_data)
                        print(f"Inserted paper: {parsed_data['paper_id']}")

# Function to read and parse Cit-HepTh.txt and create edges in the database
def create_edges_from_file(file_path, db_uploader):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("#"):
                continue
            from_node, to_node = line.strip().split()
            db_uploader.insert_citation(from_node, to_node)
            print(f"Inserted citation: {from_node} -> {to_node}")


# Function to process cit-HepTh-dates.txt and create the paper_dates table
def process_dates_file(file_path, db_uploader):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("#"):
                continue
            paper_id, publication_date = line.strip().split()
            if paper_id.startswith("11"):
                paper_id = paper_id[2:]
            db_uploader.insert_paper_date(str(int(paper_id)), publication_date)
            print(f"Inserted date for paper: {paper_id} -> {publication_date}")

# Function to compute and populate co-citation counts
def compute_and_store_co_citation_counts(db_uploader):
    # Query to compute co-citation counts
    query = """
    SELECT 
        c1.to_paper_id AS paper_1_id,
        c2.to_paper_id AS paper_2_id,
        COUNT(DISTINCT c1.from_paper_id) AS co_citation_count
    FROM 
        citations c1
    JOIN 
        citations c2 ON c1.from_paper_id = c2.from_paper_id
    WHERE 
        c1.to_paper_id < c2.to_paper_id  -- Avoid duplicate pairs
    GROUP BY 
        c1.to_paper_id, c2.to_paper_id
    """
    
    # Execute the query to compute co-citations
    db_uploader.cursor.execute(query)
    co_citation_data = db_uploader.cursor.fetchall()
    
    # Insert co-citation counts into the co_citations table
    for paper_1_id, paper_2_id, co_citation_count in co_citation_data:
        db_uploader.insert_co_citation(paper_1_id, paper_2_id, co_citation_count)

    print("Co-citation counts computed and stored successfully.")

# Usage
root_folder = "cit-HepTh-abstracts"
db_uploader = RelationalDatabaseUploader("papers.db")

# Process papers and citations
process_all_abs_files(root_folder, db_uploader)
create_edges_from_file("cit-HepTh.txt", db_uploader)

process_dates_file('cit-HepTh-dates.txt', db_uploader)

# Compute and store co-citation counts
compute_and_store_co_citation_counts(db_uploader)


db_uploader.close()
print("All data uploaded to the relational database successfully.")

Inserted paper: 9301112
Inserted paper: 9303063
Inserted paper: 9308136
Inserted paper: 9308122
Inserted paper: 9303077
Inserted paper: 9301106
Inserted paper: 9304056
Inserted paper: 9310023
Inserted paper: 9306127
Inserted paper: 9312152
Inserted paper: 9312146
Inserted paper: 9306133
Inserted paper: 9310037
Inserted paper: 9304042
Inserted paper: 9304095
Inserted paper: 9312191
Inserted paper: 9312185
Inserted paper: 9304081
Inserted paper: 9307007
Inserted paper: 9305176
Inserted paper: 9311103
Inserted paper: 9311117
Inserted paper: 9305162
Inserted paper: 9307013
Inserted paper: 9302143
Inserted paper: 9309016
Inserted paper: 9309002
Inserted paper: 9307173
Inserted paper: 9305002
Inserted paper: 9311077
Inserted paper: 9311063
Inserted paper: 9305016
Inserted paper: 9307167
Inserted paper: 9302037
Inserted paper: 9302023
Inserted paper: 9311088
Inserted paper: 9301066
Inserted paper: 9303117
Inserted paper: 9308042
Inserted paper: 9308056
Inserted paper: 9303103
Inserted paper: 

KeyboardInterrupt: 